<a href="https://colab.research.google.com/github/AditMeh/deep-learning/blob/main/Semantic_segmentation_drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from skimage import io
from skimage.color import rgb2gray
import numpy as np
import matplotlib.pyplot as plt
import os


path = "/content/drive/MyDrive/resized_dataset"

images = os.listdir(os.path.join(path + "/new_semantic_drone_dataset"))
dataset = []
stack = []

for image in images[0:10]:
    stack = []
    img = io.imread(os.path.join(os.path.join(path + "/new_semantic_drone_dataset"), image))
    for i in range(20):
        background_mask = img == i
        stack.append(background_mask)
    
    dataset.append(np.dstack([np.asarray(mask) for mask in stack]))

print([item.shape for item in dataset])
print(np.asarray(dataset).shape)


[(400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20), (400, 600, 20)]
(10, 400, 600, 20)


In [26]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPool2D, Add, Input, Conv2DTranspose, concatenate
from keras.activations import softmax


path = "/content/drive/MyDrive/resized_dataset"

def test_data_generator():
    images = os.listdir(os.path.join(path + "/new_semantic_drone_dataset"))
    for image in images:
        img = io.imread(os.path.join(os.path.join(path + "/new_semantic_drone_dataset"), image))
        stack = []
        for i in range(20):
            background_mask = img == i
            stack.append(background_mask)
        curr_item = np.dstack([np.asarray(mask) for mask in stack])
        yield tf.convert_to_tensor(curr_item, dtype="float")


def train_data_generator():
    images = os.listdir(os.path.join(path + "/new_semantic_drone_inputs"))
    for image in images:
        img = io.imread(os.path.join(os.path.join(path + "/new_semantic_drone_inputs"), image))
        yield tf.convert_to_tensor(img, dtype="float")


train_data = tf.data.Dataset.from_generator(generator=train_data_generator,
                                            output_signature=(
                                            tf.TensorSpec(shape = (400, 600, 3), dtype = tf.float32)
                                            )).batch(10, drop_remainder = False)

train_labels = tf.data.Dataset.from_generator(generator=test_data_generator,
                                            output_signature=(
                                            tf.TensorSpec(shape = (400, 600, 20), dtype = tf.float32)
                                            )).batch(10, drop_remainder = False)

In [9]:
print(next(train_data.as_numpy_iterator()).shape)

print(next(train_labels.as_numpy_iterator()).shape)

(10, 400, 600, 3)
(10, 400, 600, 20)


In [45]:
# Define model:

class ConvBlock(keras.layers.Layer):
    def __init__(self, filters):
        super().__init__()

        self.conv1 = Conv2D(filters, kernel_size = (3,3),
                            strides = 1,
                            padding = "same",
                            activation = "relu",
                            data_format = "channels_last",
                            kernel_initializer = "GlorotNormal")

        self.conv2 = Conv2D(filters, kernel_size = (3,3),
                            strides = 1,
                            padding = "same",
                            activation = "relu",
                            data_format = "channels_last",
                            kernel_initializer = "GlorotNormal")
    def call(self, X):
        conv1 = self.conv1(X)
        conv2 = self.conv2(conv1)
        return conv2

def build_unet():
        input = Input(shape = (400, 600, 3))


        # Downsampling path
        conv1 = ConvBlock(64)(input)

        pool1 = MaxPool2D(pool_size= (2,2))(conv1)

        conv2 = ConvBlock(128)(pool1)

        pool2 = MaxPool2D(pool_size= (2,2))(conv2)

        conv3 = ConvBlock(256)(pool2)

        pool3 = MaxPool2D(pool_size= (2,2))(conv3)

        conv4 = ConvBlock(512)(pool3)

        pool4 = MaxPool2D(pool_size= (2,2))(conv4)

        conv5 = ConvBlock(1024)(pool4)

        #Upsampling path

        upConv1 = Conv2DTranspose(filters=512, kernel_size=(2,2), padding = "same")(conv5)
        concat1 = concatenate([upConv1, conv4], axis=3)
        conv6 = Conv2D(512)(concat1)

        upConv2 = Conv2DTranspose(filters=256, kernel_size=(2,2), padding = "same")(conv6)
        concat2 = concatenate([upConv2, conv3], axis=3)
        conv7 = Conv2D(256)(concat2)

        upConv3 = Conv2DTranspose(filters=128, kernel_size=(2,2), padding = "same")(conv7)
        concat3 = concatenate([upConv3, conv2], axis=3)
        conv8 = Conv2D(64)(concat3)

        upConv4 = Conv2DTranspose(filters=64, kernel_size=(2,2), padding = "same")(conv8)
        concat4 = concatenate([upConv4, conv1], axis=3)
        conv9 = Conv2D(64)(concat4)

        conv10 = Conv2D(20, kernel_size = (3,3),
                        padding = "same",
                        activation = "relu",
                        data_format = "channels_last",
                        kernel_initializer = "GlorotNormal")(conv9)
        
        final = softmax(conv10, axis = 3)

        model = Model(input = inputs, output = final)

        model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

        return model




In [46]:
build_unet()

ValueError: ignored

In [47]:
inp = np.ones(shape=(1, 400, 600, 3))

conv1 = ConvBlock(64)

pool1 = MaxPool2D(pool_size= (2,2))

conv2 = ConvBlock(128)

pool2 = MaxPool2D(pool_size= (2,2))

conv3 = ConvBlock(256)

pool3 = MaxPool2D(pool_size= (2,2))

conv4 = ConvBlock(512)

pool4 = MaxPool2D(pool_size= (2,2))

conv5 = ConvBlock(1024)


print(conv5(pool4(conv4(pool3(conv3(pool2(conv2(pool1(conv1(inp))))))))).shape)


print(conv1(inp).shape)

(1, 25, 37, 1024)
(1, 400, 600, 64)
